# DATA EXPLORATION

# FRAUD DETECTION

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load the dataset
data = pd.read_csv('simulated_transaction_2024.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.shape 

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
# Visualize missing values as a heatmap
plt.figure(figsize=(10,6))
sns.heatmap(data.isnull(), cbar=False)
plt.title('Heatmap of Missing Values in Data')
plt.show()

### This map is not giving clear missing values, as we have a large dataset, lets proceed normally

In [ ]:
data.isnull().sum()

In [ ]:
# Calculate the percentage of null values for each column
data.isnull().mean() * 100

### i want to to check the number of transactions of other users, to get more details

In [ ]:
# 'Account No' and number of transactions for each account
transactions_per_user = data.groupby('Account No').size()

transactions_count = transactions_per_user.reset_index(name='Number of Transactions')
transactions_count.head()

In [ ]:
# finding average number of transactions of all users
#avg_transaction_per_user = 
transactions_per_user.mean()
#avg_transaction_per_user

### we have 224 missing values in account number, i think we can create a random account number

In [ ]:
# Generating unique account number to remove null values
existing_acc_numbers = pd.to_numeric(data['Account No'], errors='coerce').dropna().unique()
new_acc_number = int(max(existing_acc_numbers) + 1)

# Replace null values in the "Account No" column with the new account number
data['Account No'].fillna(new_acc_number, inplace=True)

# Verifyong that there are no null values in the "Account No" column
verification_result = data['Account No'].isnull().sum()

new_acc_number, verification_result

In [ ]:
print(data[(data['Account No'] == 999752673)  ])

## what is done till now:
- data exploration, using functions like shape(),info(),isnull().sum(),describe(), dtypes().
- imputing null values in account number: by the taking the the average transaction count (taking mean),which is 236, and our missing values count is 224. So, I decided to generate a new account number instead of removing them.


In [ ]:
## float to integer
data['Account No'] = data['Account No'].astype(int)

In [ ]:
# Convert 'Third Party Account No' to nullable integer type
data['Third Party Account No'] = data['Third Party Account No'].astype('Int64')

In [ ]:
# Create a new 'Third Party' column
data['Third Party'] = data['Third Party Name'].combine_first(data['Third Party Account No'])

In [ ]:
# Drop the original 'Third Party Account No' and 'Third Party Name' columns
data.drop(columns=['Third Party Account No', 'Third Party Name'], inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data['Account No'].nunique() ## total no.of users

In [ ]:
## find unique and similarities

In [ ]:
data.nunique()

## filling missing values in date and timestamp :
##### 1. Date and Timestamp
- Method Used: Forward Fill
- Why?
- Both the Date and Timestamp fields are critical for maintaining the sequence and continuity in time   series data.
- Forward fill is appropriate here because it assumes that the missing date or timestamp can reasonably be replaced by the last known value, which is a common practice in time-sensitive data to maintain alignment in time series analysis.
- This method helps avoid introducing artificial discontinuities in the time sequence, which could occur if we interpolated times or used backward fill.

In [ ]:
# Fill missing Date and Timestamp
data['Date'] = data['Date'].fillna(method='ffill')
data['Timestamp'] = data['Timestamp'].fillna(method='ffill')

In [ ]:
# Convert the 'not_happened_yet_date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

### i want to impute the missing values in amount and balance column  as well, but I have observed some abnormalities in account transactions

In [ ]:
data.hist( figsize = (12,8) )
plt.show() ## to see the distributions of data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Plot the transaction frequency by hour
plt.figure(figsize=(8, 3))
plt.hist(pd.to_datetime(data['Timestamp']).dt.hour, bins=24, edgecolor='black')
plt.title('Transaction Frequency by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Transactions')
plt.xticks(range(24))  # Ensures ticks for every hour
plt.tight_layout()  # Adjust layout to fit the figure size
plt.show()

# Plot the total credited amount to third parties by hour
plt.figure(figsize=(12, 6))
data.loc[data['Amount'] < 0].groupby(pd.to_datetime(data['Timestamp']).dt.hour)['Amount'].apply(lambda x: abs(x).sum()).plot(
    kind='bar', color='darkgreen', edgecolor='black')
plt.title('Total Credited Amount to Third Parties by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Credited Amount ($)')
plt.xticks(range(24))  # Ensures ticks for every hour
plt.grid(axis='y', linestyle='--', linewidth=0.7)
plt.tight_layout()  # Adjust layout to fit the figure size
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Plot the transaction frequency by half-hour intervals
plt.figure(figsize=(10, 4))
plt.hist(pd.to_datetime(data['Timestamp']).dt.round('30min').dt.strftime('%H:%M'), bins=48, edgecolor='black')
plt.title('Transaction Frequency by Half-Hour Interval')
plt.xlabel('Half-Hour Interval')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to fit the figure size
plt.show()

# Plot the total credited amount to third parties by half-hour intervals
plt.figure(figsize=(12, 6))
data.loc[data['Amount'] < 0].groupby(pd.to_datetime(data['Timestamp']).dt.round('30min').dt.strftime('%H:%M'))['Amount'].apply(lambda x: abs(x).sum()).plot(
    kind='bar', color='darkgreen', edgecolor='black')
plt.title('Total Credited Amount to Third Parties by Half-Hour Interval')
plt.xlabel('Half-Hour Interval')
plt.ylabel('Total Credited Amount ($)')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.grid(axis='y', linestyle='--', linewidth=0.7)
plt.tight_layout()  # Adjust layout to fit the figure size
plt.show()


## First plot:
- transaction frequency by hour (activity at certain times of the day). 
- It appears that there's a peak in transactions towards the end of the day, around 23:00. 
## second plot:
- plot shows the total credited amount to third parties by hour.
- presents the absolute values of the negative amounts to reflect the spending positively.

In [ ]:
import matplotlib.pyplot as plt

# Filter data for transactions where amount is negative (indicating spending)
spending_data = data[data['Amount'] < 0]

# Group by 'Account No' and calculate total spending for each user
total_spending_per_user = spending_data.groupby('Account No')['Amount'].sum()

# Get the top 5 users with the highest total spending
top_5_users = total_spending_per_user.nlargest(5)

# Print the top 5 users and their total spending
print("Top 5 Users with Highest Total Spending:")
print(top_5_users)

# Plot spending patterns for the top 5 users
plt.figure(figsize=(12, 8))
for user_id in top_5_users.index:
    user_transactions = spending_data[spending_data['Account No'] == user_id]
    plt.scatter(user_transactions['Timestamp'], user_transactions['Amount'], label=f'User {user_id}')

plt.title('Spending Patterns of Top 5 Users')
plt.xlabel('Timestamp')
plt.ylabel('Amount')
plt.xticks(rotation=45)
plt.legend()
plt.show()


In [ ]:
data

## how to impute amount and balance column 

## analyze the relationship between transaction direction (debit/credit) and balance:
1. Segment the Data
2. Statistical Summary
3. Visual Analysis

In [ ]:
# Identify debits and credits
debits = data[data['Amount'] < 0]
credits = data[data['Amount'] > 0]

In [ ]:
# Calculate statistics for debits and credits
debit_stats = debits['Balance'].describe()
credit_stats = credits['Balance'].describe()

print("Debit Statistics:\n", debit_stats)
print("\nCredit Statistics:\n", credit_stats)

In [ ]:
# Sort data by Date and Account No
data.sort_values(by=['Account No', 'Date'], inplace=True)

# Forward fill missing Balance and Amount
data['Balance'] = data.groupby('Account No')['Balance'].fillna(method='ffill')
data['Amount'] = data.groupby('Account No')['Amount'].fillna(method='ffill')

In [ ]:
# Optional: Plot to visualize imputation effects
import matplotlib.pyplot as plt

# Filter for a specific account to visualize
sample_account = data[data['Account No'] == 999752673]  # Replace some_account_number with an actual number

plt.figure(figsize=(14, 7))
plt.subplot(2, 1, 1)
plt.plot(sample_account['Date'], sample_account['Balance'], marker='o', linestyle='-')
plt.title('Balance Over Time')
plt.xlabel('Date')
plt.ylabel('Balance')

plt.subplot(2, 1, 2)
plt.plot(sample_account['Date'], sample_account['Amount'], marker='o', linestyle='-')
plt.title('Transaction Amount Over Time')
plt.xlabel('Date')
plt.ylabel('Amount')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Label the transactions as debit or credit
data['Transaction Type'] = ['Credit' if amt > 0 else 'Debit' for amt in data['Amount']]

# Plotting the distribution of Balances for debits and credits
plt.figure(figsize=(6,3))
sns.boxplot(x='Transaction Type', y='Balance', data=data)
plt.title('Balance Distribution for Debits and Credits')
plt.xlabel('Transaction Type')
plt.ylabel('Balance')
plt.show()

In [ ]:
# Interpolate missing values for Balance and Amount
#data['Balance'] = data['Balance'].interpolate(method='linear')
#data['Amount'] = data['Amount'].interpolate(method='linear')

In [ ]:
# nearest integer/ rounding
data['Balance'] = data['Balance'].round().astype(int)
data['Amount'] = data['Amount'].round().astype(int)

In [ ]:
# Calculate statistics for debits and credits
debit_stats = debits['Balance'].describe()
credit_stats = credits['Balance'].describe()

In [ ]:
debit_stats

In [ ]:
credit_stats

## Transactions that are several standard deviations away from the mean might be considered suspicious:
- Transactions that are several standard deviations away from the mean might be considered suspicious:

In [ ]:
# Calculate the mean and standard deviation
mean_amount = data['Amount'].mean()
std_amount = data['Amount'].std()

# Identify outliers as transactions that are more than 3 standard deviations from the mean
outliers = data[np.abs(data['Amount'] - mean_amount) > 3 * std_amount]

# Display outliers
outliers[['Date','Timestamp', 'Amount', 'Account No', 'Third Party', 'Transaction Type']]

## transactions occurring repeatedly at the same time to the same third party, which could indicate automated or scripted fraud:
- more than 5 times

In [ ]:
# Group by account number, third party, and hour to see repetitive patterns
repetitive_patterns = data.groupby(['Account No', 'Third Party', 'Timestamp']).size().reset_index(name='Count')

# Filter for patterns that occur more than a threshold, e.g., more than 5 times
suspicious_patterns = repetitive_patterns[repetitive_patterns['Count'] > 5]

# Display suspicious repetitive patterns
suspicious_patterns


## Visualisations

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot transaction amounts for outliers
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Date', y='Amount', hue='Transaction Type', data=outliers)
plt.title('Outlier Transactions Over Time')
plt.xlabel('Date')
plt.ylabel('Transaction Amount')
plt.show()

In [ ]:
# Convert 'Date' and 'Timestamp' into a single datetime column for easier manipulation
#data['Timestamp'] = pd.to_datetime(data['Date'] + ' ' + data['Timestamp'], errors='coerce')

# Calculate mean and standard deviation for each account without storing them as columns
mean_amounts = data.groupby('Account No')['Amount'].transform('mean')
std_amounts = data.groupby('Account No')['Amount'].transform('std')

# Identify high-value transactions
high_value = data['Amount'] > (mean_amounts + 3 * std_amounts)

# Detect frequent transactions in short times by counting the transactions per minute for each account
frequent_transactions = data.groupby(['Account No', data['Timestamp'].dt.floor('min')])['Amount'].transform('size') > 3

# Check for transactions that are round figures
round_figure_transaction = data['Amount'].mod(1000) == 0

# Combine the criteria to filter potential fraudulent transactions
suspicious_transactions = data[high_value | frequent_transactions | round_figure_transaction]

# Display the suspicious transactions
print(suspicious_transactions[['Date', 'Timestamp', 'Account No', 'Amount', 'Third Party', 'Transaction Type']])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot transaction amounts for potential outliers
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Date', y='Amount', hue='Transaction Type', style='Day of Week', data=data)
plt.title('Transaction Amounts Over Time')
plt.xlabel('Date')
plt.ylabel('Transaction Amount')
plt.show()


In [ ]:
# Create pairplot for selected columns to visualize relationships and distributions
sns.pairplot(data[['Amount', 'Balance', 'Transaction Direction']], hue='Transaction Direction', plot_kws={'alpha': 0.5})
plt.suptitle('Pairplot of Transactions', size=16, y=1.02)
plt.show()

In [ ]:
##

In [ ]:
# Create individual scatter plots instead of a pairplot to avoid issues with KDE and non-numeric data types
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot of Amount vs. Balance
sns.scatterplot(x='Amount', y='Balance', hue='Transaction Direction', data=data, ax=axs[0], alpha=0.5)
axs[0].set_title('Scatter Plot of Amount vs. Balance')
axs[0].set_xlabel('Amount')
axs[0].set_ylabel('Balance')

# Scatter plot of Amount vs. Hour of Transaction
sns.scatterplot(x='Amount', y='Timestamp', hue='Transaction Direction', data=data, ax=axs[1], alpha=0.5)
axs[1].set_title('Scatter Plot of Amount vs. Hour of Transaction')
axs[1].set_xlabel('Amount')
axs[1].set_ylabel('Hour of Transaction')

plt.tight_layout()
plt.show()

## Monthly Banking Activity: Insights from Total Spent, Total Credited, and Transaction Count

In [ ]:

# Using 'Grouper' to group by month directly

monthly_data = data.groupby(['Account No', pd.Grouper(key='Date', freq='M')]).agg(
    Total_Spent=pd.NamedAgg(column='Amount', aggfunc=lambda x: x[x < 0].sum()),
    Total_Credited=pd.NamedAgg(column='Amount', aggfunc=lambda x: x[x >= 0].sum()),
    Transaction_Count=pd.NamedAgg(column='Amount', aggfunc='count')
).reset_index()

# Converting 'Date' to 'Year-Month' format for easier reading
monthly_data['Date'] = monthly_data['Date'].dt.to_period('M')

# Display the first few rows of the modified monthly statistics
monthly_data.head(5)


In [ ]:
monthly_data.shape

In [ ]:
data['Account No'].unique().shape

## checking all the users spending evry month or not

In [ ]:
# To ensure each account's activity is accounted for in every expected month of the dataset, 
# we'll check which months each account has transactions and list any missing months.

# First, get the full range of months from the dataset
full_date_range = pd.period_range(data['Date'].min(), data['Date'].max(), freq='M')

# Dictionary to store missing months for each account
missing_months_dict = {}

# Iterating over each account
for account, group in data.groupby('Account No'):
    present_months = group['Date'].dt.to_period('M').unique()
    missing_months = full_date_range[~full_date_range.isin(present_months)]
    if len(missing_months) > 0:
        missing_months_dict[account] = missing_months

# Create a list to store tuples of account number and missing month
missing_months_list = []

# Iterate over the dictionary and append tuples to the list
for account, missing_months in missing_months_dict.items():
    for month in missing_months:
        missing_months_list.append((account, month))

# Create a DataFrame from the list
missing_months_df = pd.DataFrame(missing_months_list, columns=['Account No', 'Missing Month'])

# Display the DataFrame
print(missing_months_df)


In [ ]:
## no.of transactions on each day..smtwthfss..
# find any trends
## his status at the end of month- whether he is in debt or credit

In [ ]:
data

## fraud detection using transactional data

In [ ]:
suspicious_transactions

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming suspicious_transactions is already defined
# and data['Timestamp'] has been created and is the datetime combination of 'Date' and 'Timestamp'

# 1. Scatter Plot of Transaction Amounts Over Time
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Timestamp', y='Amount', data=suspicious_transactions, hue='Transaction Direction', style='Transaction Direction')
plt.title('Suspicious Transactions Over Time')
plt.xlabel('Timestamp')
plt.ylabel('Transaction Amount')
plt.xticks(rotation=45)
plt.legend(title='Transaction Type')
plt.tight_layout()
plt.show()

# 2. Histogram of Transaction Amounts
plt.figure(figsize=(10, 6))
sns.histplot(suspicious_transactions['Amount'], bins=30, kde=False, color='red')
plt.title('Distribution of Suspicious Transaction Amounts')
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming suspicious_transactions is already defined
# and data['Timestamp'] has been created and is the datetime combination of 'Date' and 'Timestamp'

# Extract more features from 'Timestamp' if needed
suspicious_transactions['Hour of Day'] = suspicious_transactions['Timestamp'].dt.hour
suspicious_transactions['Day of Week'] = suspicious_transactions['Timestamp'].dt.dayofweek

# Select columns to include in the pairplot
# Including 'Amount', 'Hour of Day', and 'Day of Week'
# If 'Transaction Direction' is categorical, it can be used as a hue
plot_data = suspicious_transactions[['Amount', 'Hour of Day', 'Day of Week', 'Transaction Direction']]

# Create the pairplot
sns.pairplot(plot_data, hue='Transaction Direction', diag_kind='kde', plot_kws={'alpha': 0.6, 's': 60, 'edgecolor': 'k'}, height=3)
plt.suptitle('Pairplot of Suspicious Transactions')
plt.show()
